In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scanpy.api as sc
import pandas as pd
import h5py
import os
import time
import itertools
from sklearn.decomposition import PCA

## Network
from community import community_louvain
import networkx as nx
import matplotlib.pyplot as plt

import scanpy.api as sc
import sceb.data_loader as dl 
import sceb.scdd as sd
import csv

from sklearn.metrics import roc_curve

%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/martin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
output_folder = '/home/martin/single_cell_eb/figures/figure_gene_network'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [3]:
## load the data 
data = dl.load_10x_4k()
data.var_names_make_unique()
sc.pp.filter_genes(data,min_counts=0.0001*data.shape[0])
Nc,G,Nr,Nr_bar = sd.get_info(data)

## filter and calculate PC
size_factor = sd.dd_size_factor(data,verbose=False)
M_ml,M_dd = sd.dd_1d_moment(data,size_factor=size_factor)
var_dd = sd.M_to_var(M_dd)
gene_list = np.array(data.var_names)[var_dd>0.1]
print(len(gene_list))
data = data[:,list(gene_list)]

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
## Nc=4340, G=19773, Nr=4349.04, Nr_bar=0.22
#time start: 0.0s
n_cell=4340, n_gene=19773
#total: 0.28s
791


In [4]:
# calculate PC
mean_ml,cov_ml,PC_ml  = sd.ml_covariance(data,size_factor=size_factor)
mean_dd,cov_dd,PC_dd  = sd.dd_covariance(data,size_factor=size_factor,PC_prune=False)

#time start: 0.0s
n_cell=4340, n_gene=791, Nr=3224.3
#total: 3.38s
#time start: 0.0s
n_cell=4340, n_gene=791, Nr=3224.3
#total: 3.18s


In [76]:
# write the data 
def write_NET_csv(knn,gnames,outfile):
    adj_csv = []
    adj_csv= [['']+[i for i in gnames]]
    for i in range(knn.shape[0]):
        adj_csv+= [ [gnames[i]]+[j for j in knn[:,i]]  ]

    myFile = open(outfile, 'w')
    with myFile:
        writer = csv.writer(myFile,delimiter=';')
        writer.writerows(adj_csv)

## write the network file
fil=0.8
outfile = output_folder + '/network_data.csv'
temp = (PC_dd>0.8).astype(int)
np.fill_diagonal(temp,0)
write_NET_csv(temp,gene_list,outfile)